# Auto Scout Project

**Author:** Mark Joslin

**GitHub:** [link](https://github.com/mjoslin-ai/Magnimind-Project-Auto-Scout)

**Phase 2:** Missing Value Analysis

## Table of Contents

* Objective
* Libraries
* Import Data
* Missing Value Analysis
* Export Data

## Objective

Clean and preprocess a 2019 online vehicle dataset (≈15,919 rows, 54 columns, 9 car models) containing many inconsistent, broken, and messy values, so it becomes suitable for machine learning. The project is broken up into four main phases: 

1. Data Cleaning: 
    * Removing broken, irrelevant, or redundant columns
    * Creating new columns with clean, meaningful, and usable values
2. Missing Value Analysis:
    * Filling in the gabs where data is missing
    * With techniques such as mean, median, or mode
3. Outlier Analysis:
    * Outliers are detected and visualized using boxplots and histograms.
    * Rather than removing all outliers (since extreme but valid values are expected for certain car models, e.g. luxury vehicles with high prices or powerful engines), the focus is on identifying and addressing only clear misprints or logically inconsistent entries.
4. Dummy Variable Analysis:
    * Categorical and multi-label columns (e.g. Comfort_Convenience, Extras) are converted to numeric format to create binary indicator variables suitable for machine learning.
    * Key techniques include pd.get_dummies() and str.get_dummies(sep=",")

This notebook covers Phase 2 Missing Value Analysis with input from phase1_cleaned_data.csv (15919 rows, 35 columns) and output phase2_cleaned_data.csv.

## Libraries

In [530]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

## Import Data

In [531]:
df_org = pd.read_csv("phase1_cleaned_data.csv")
df = df_org.copy()
df.head(3).T

,0,1,2
make_model,Audi A1,Audi A1,Audi A1
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,56013.0,80000.0,83450.0
previous_owners,2.0,NaN,1.0
next_inspection,2021-06,NaN,NaN
inspection_new,Yes,NaN,NaN
body_color,Black,Red,Black
paint_type,metallic,NaN,metallic


In [532]:
missing_df = df.isnull().sum().reset_index()
missing_df.columns = ['column', 'missing_count']
missing_df = missing_df[missing_df['missing_count'] > 0].reset_index(drop=True)
print(missing_df)

                  column  missing_count
0              body_type             60
1                    vat           4513
2                     km           1024
3        previous_owners           6640
4        next_inspection          12384
5         inspection_new          11987
6             body_color            597
7             paint_type           5772
8            nr_of_doors            212
9            nr_of_seats            977
10             cylinders           5680
11           drive_chain           6858
12        emission_class           3628
13   comfort_convenience            920
14   entertainment_media           1374
15                extras           2962
16       safety_security            982
17                 gears           4712
18       displacement_cc            213
19             age_years           1597
20                 hp_kw             88
21             condition              2
22       warranty_months          11066
23       upholstery_type           4871


## Missing Value Analysis

### body_type

In [533]:
df['body_type'].value_counts(dropna=False)

body_type
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

NaNs are filled in with make_model

In [534]:
bt = df['body_type'].fillna(df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0]))

In [535]:
bt.value_counts(dropna=False)

body_type
Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

In [536]:
df['body_type'] = bt

### condition

In [537]:
df['condition'].value_counts(dropna=False)

condition
Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: count, dtype: int64

With only two NaNs, they are most likely 'Used' and little risk of distortion if not.

In [538]:
c = df['condition'].fillna(df['condition'].mode()[0])

In [539]:
c.value_counts(dropna=False)

condition
Used              11098
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: count, dtype: int64

In [540]:
df['condition'] = c

### vat

In [541]:
df['vat'].value_counts(dropna=False)

vat
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: count, dtype: int64

In [542]:
v = df['vat'].fillna('VAT nondeductible')

In [543]:
v.value_counts(dropna=False)

vat
VAT deductible       10980
VAT nondeductible     4513
Price negotiable       426
Name: count, dtype: int64

In [544]:
df['vat'] = v

### age_years

In [545]:
df['age_years'].value_counts(dropna=False)

age_years
1.0    4522
3.0    3674
2.0    3273
0.0    2853
NaN    1597
Name: count, dtype: int64

In [546]:
df.loc[df['age_years'].isna(), 'condition'].value_counts(dropna=False)

condition
New               1547
Used                36
Pre-registered       6
Demonstration        5
Employee's car       3
Name: count, dtype: int64

NaNs are likely 0 or less than 1 year

In [547]:
df['age_years'] = df['age_years'].fillna(0.0)

### km

In [548]:
df['km'].value_counts(dropna=False)

km
10.0      1045
NaN       1024
1.0        367
5.0        170
50.0       148
          ... 
3353.0       1
2860.0       1
2864.0       1
1506.0       1
57.0         1
Name: count, Length: 6690, dtype: int64

Group by make_model, body_type, and age_years, and fill NaNs with median value

In [549]:
k = df['km'].fillna(df.groupby(['make_model', 'body_type', 'age_years'])['km'].transform(lambda x: x.median()))

In [550]:
k.isna().sum()

np.int64(4)

In [551]:
k = k.fillna(df.groupby(['make_model', 'age_years'])['km'].transform(lambda x: x.median()))

In [552]:
k.isna().sum()

np.int64(0)

In [553]:
k.value_counts(dropna=False)

km
10.0      1280
1.0        367
1000.0     224
100.0      197
5.0        174
          ... 
3353.0       1
2860.0       1
2864.0       1
1506.0       1
57.0         1
Name: count, Length: 6696, dtype: int64

In [554]:
df['km'] = k

### previous_owners

In [555]:
df['previous_owners'].value_counts(dropna=False)

previous_owners
1.0    8294
NaN    6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: count, dtype: int64

In [556]:
df.loc[df['previous_owners'].isna(), ['condition', 'age_years']].value_counts(dropna=False)

condition       age_years
New             0.0          1469
Used            3.0          1221
                2.0          1173
                1.0          1153
                0.0           465
Pre-registered  0.0           454
Demonstration   0.0           242
Employee's car  1.0           221
                2.0            71
Pre-registered  1.0            62
Employee's car  0.0            56
Demonstration   1.0            34
Employee's car  3.0             8
Demonstration   2.0             5
Pre-registered  2.0             3
New             1.0             2
Demonstration   3.0             1
Name: count, dtype: int64

In [557]:
po = df['previous_owners'].fillna(df.groupby(['condition', 'age_years'])['previous_owners'].transform(lambda x: x.median()))

In [558]:
po.isna().sum()

np.int64(0)

In [559]:
po.value_counts(dropna=False)

previous_owners
1.0    13465
0.0     1657
2.0      778
3.0       17
4.0        2
Name: count, dtype: int64

In [560]:
df['previous_owners'] = po

### next_inspection

In [561]:
df['next_inspection'].value_counts(dropna=False)

next_inspection
NaN        12384
2021-06      471
2021-03      210
2021-05      180
2021-04      171
           ...  
2017-12        1
2022-12        1
1921-01        1
2024-03        1
2022-11        1
Name: count, Length: 78, dtype: int64

In [562]:
df.loc[df['next_inspection'].isna(), 'age_years'].value_counts(dropna=False)

age_years
0.0    3607
1.0    3307
3.0    2884
2.0    2586
Name: count, dtype: int64

In EU, next inspection follows the periodic technical inspections (PTI) of vehicles 4 years after first registration

In [563]:
ni = df['next_inspection'].fillna((2019 - df['age_years'] + 4).round(0).astype(int).astype(str) + '-01')

ni = pd.to_datetime(ni, errors='coerce').dt.to_period('M')

In [564]:
ni.isna().sum()

np.int64(0)

In [565]:
ni.value_counts(dropna=False)

next_inspection
2023-01    3627
2022-01    3414
2020-01    2947
2021-01    2718
2021-06     471
           ... 
2017-12       1
2022-12       1
1921-01       1
2024-03       1
2022-11       1
Freq: M, Name: count, Length: 77, dtype: int64

In [566]:
df['next_inspection'] = ni

### inspection_new

In [567]:
df['inspection_new'].value_counts(dropna=False)

inspection_new
NaN    11987
Yes     3932
Name: count, dtype: int64

In [568]:
df['inspection_new'] = df['inspection_new'].fillna('No')

### body_color

In [569]:
df['body_color'].value_counts(dropna=False)

body_color
Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: count, dtype: int64

In [570]:
bc = df['body_color'].fillna(df.groupby(['make_model', 'body_type'])['body_color'].transform(lambda x: x.mode()[0] if not x.mode().empty else df['body_color'].mode()[0]))

In [571]:
bc.value_counts(dropna=False)

body_color
Black     3944
Grey      3836
White     3469
Silver    1647
Blue      1431
Red        957
Brown      289
Green      158
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: count, dtype: int64

In [572]:
df['body_color'] = bc

### paint_type

In [573]:
df['paint_type'].value_counts(dropna=False)

paint_type
metallic       9794
NaN            5772
uni/basic       347
perl effect       6
Name: count, dtype: int64

In [574]:
df.loc[df['paint_type'].isna(), ['make_model', 'body_type']].value_counts(dropna=False)

make_model      body_type    
Audi A3         Sedans           1015
Renault Clio    Sedans            511
Audi A1         Sedans            490
Opel Corsa      Compact           486
Opel Insignia   Station wagon     450
Opel Corsa      Sedans            358
Opel Astra      Station wagon     340
                Sedans            333
Audi A1         Compact           311
Opel Insignia   Sedans            310
Renault Espace  Van               292
Renault Clio    Compact           201
                Station wagon     164
Audi A3         Station wagon     113
Opel Astra      Compact            87
Audi A3         Compact            73
Renault Espace  Station wagon      28
Renault Clio    Transporter        26
Opel Astra      Other              24
Opel Insignia   Other              23
Opel Corsa      Other              20
Renault Clio    Other              20
Renault Espace  Transporter        18
Renault Duster  Off-Road           17
Opel Corsa      Coupe               9
Renault Espace  Seda

In [575]:
df['paint_type'].mode()[0]

'metallic'

In [576]:
pt = df['paint_type'].fillna(df.groupby(['make_model', 'body_type'])['paint_type'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [577]:
pt.isna().sum()

np.int64(11)

In [578]:
pt = pt.fillna(df.groupby(['make_model'])['paint_type'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [579]:
pt.isna().sum()

np.int64(0)

In [580]:
pt.value_counts(dropna=False)

paint_type
metallic       15523
uni/basic        390
perl effect        6
Name: count, dtype: int64

In [581]:
df['paint_type'] = pt

### nr_of_doors

In [583]:
df['nr_of_doors'].value_counts(dropna=False)

nr_of_doors
5.0    11575
4.0     3079
3.0      832
2.0      219
NaN      212
1.0        1
7.0        1
Name: count, dtype: int64

In [584]:
nod = df['nr_of_doors'].fillna(df.groupby(['body_type'])['nr_of_doors'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [585]:
nod.isna().sum()

np.int64(0)

In [587]:
df['nr_of_doors'] = nod

### nr_of_seats

In [589]:
df['nr_of_seats'].value_counts(dropna=False)

nr_of_seats
5.0    13336
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: count, dtype: int64

In [590]:
nos = df['nr_of_seats'].fillna(df.groupby(['body_type', 'make_model'])['nr_of_seats'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [591]:
nos.isna().sum()

np.int64(6)

In [592]:
nos = nos.fillna(df.groupby(['body_type'])['nr_of_seats'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [593]:
nos.isna().sum()

np.int64(0)

In [594]:
df['nr_of_seats'] = nos

### cylinders

In [596]:
df['cylinders'].value_counts(dropna=False)

cylinders
4.0    8105
NaN    5680
3.0    2104
5.0      22
6.0       3
8.0       2
2.0       2
1.0       1
Name: count, dtype: int64

In [597]:
cy = df['cylinders'].fillna(df.groupby(['make_model', 'body_type'])['cylinders'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [598]:
cy.isna().sum()

np.int64(7)

In [599]:
cy = cy.fillna(df.groupby(['make_model'])['cylinders'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [600]:
cy.isna().sum()

np.int64(1)

In [ ]:
cy = cy.fillna(cy.mode()[0])

In [602]:
df['cylinders'] = cy

### drive_chain

In [604]:
df['drive_chain'].value_counts(dropna=False)

drive_chain
front    8886
NaN      6858
4WD       171
rear        4
Name: count, dtype: int64

In [605]:
dc = df['drive_chain'].fillna(df.groupby(['make_model', 'body_type'])['drive_chain'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [606]:
dc.isna().sum()

np.int64(12)

In [607]:
dc = dc.fillna(df.groupby(['make_model'])['drive_chain'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [608]:
dc.isna().sum()

np.int64(0)

In [609]:
df['drive_chain'] = dc

### emission_class

In [611]:
df['emission_class'].value_counts(dropna=False)

emission_class
Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: count, dtype: int64

In [612]:
ec = df['emission_class'].fillna(df.groupby(['fuel', 'age_years'])['emission_class'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [613]:
ec.isna().sum()

np.int64(0)

In [614]:
df['emission_class'] = ec

### comfort_convenience

In [616]:
df['comfort_convenience'].value_counts(dropna=False)

comfort_convenience
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                         

In [617]:
cc = df['comfort_convenience'].fillna(df.groupby(['make_model', 'body_type', 'age_years', 'price'])['comfort_convenience'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [618]:
cc.isna().sum()

np.int64(612)

In [619]:
cc = cc.fillna(df.groupby(['make_model', 'body_type', 'age_years'])['comfort_convenience'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [620]:
cc.isna().sum()

np.int64(17)

In [621]:
cc = cc.fillna(df.groupby(['make_model', 'body_type'])['comfort_convenience'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA))

In [622]:
cc.isna().sum()

np.int64(0)

In [623]:
df['comfort_convenience'] = cc

### entertainment_media

In [625]:
df['entertainment_media'].value_counts(dropna=False)

entertainment_media
NaN                                                                                                 1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                          1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                       982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                             783
On-board computer,Radio                                                                              487
                                                                                                    ... 
CD player,Hands-free equipment,On-board computer,Radio,Sound system,Television                         1
Digital radio,Hands-free equipment,On-board computer,Radio,Sound system                                1
Digital radio,USB                                                                                      1
CD player,Digital radio,Hands-free 

## Export Data

In [624]:
missing_df = df.isnull().sum().reset_index()
missing_df.columns = ['column', 'missing_count']
missing_df = missing_df[missing_df['missing_count'] > 0].reset_index(drop=True)
print(missing_df)

                  column  missing_count
0    entertainment_media           1374
1                 extras           2962
2        safety_security            982
3                  gears           4712
4        displacement_cc            213
5                  hp_kw             88
6        warranty_months          11066
7        upholstery_type           4871
8       upholstery_color           6821
9              weight_kg           6974
10  combined_l_per_100km           2033
11      city_l_per_100km           2436
12   country_l_per_100km           2376
13    emissions_g_co2/km           2436
